In [1]:
import pandas as pd
import numpy as np
import psycopg2
from utils.db_connect import create_db_connection
from sqlalchemy import create_engine

In [2]:
conn = create_db_connection(host='localhost')

count sensors

tickets

tickets correct

In [3]:
# count tickets for each trip
conn.rollback()
query = """
SELECT 
    COUNT(fare) as count_fare,
    trip_id,
    DATE_TRUNC('day', timestamp) as day,
    MAX(timestamp) as timestamp
FROM 
    raw_tickets 
GROUP BY 
    trip_id, 
    DATE_TRUNC('day', timestamp),
    DATE_TRUNC('month', timestamp),
    DATE_TRUNC('year', timestamp)
ORDER BY 
    day
"""

tickets =pd.read_sql(query, conn)
tickets.head()

,count_fare,trip_id,day,timestamp
0,212,0004247422025030620250612,2025-05-16,2025-05-16 18:31:00
1,10,0004246692025030620250612,2025-05-16,2025-05-16 14:28:00
2,6,0004206572024090920250612,2025-05-16,2025-05-16 13:17:00
3,80,0004229852024090920250612,2025-05-16,2025-05-16 16:01:00
4,91,0004199532024090920250612,2025-05-16,2025-05-16 08:28:00


In [10]:
conn.rollback()

query = """
SELECT *
FROM trips
"""
trips = pd.read_sql(query, conn)
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,607,0000001432024090920250612,0004187192024090920250612,Sorni,False,None,NaN
1,607,0000001442024090920250612,0004187202024090920250612,Pressano Piazza,False,None,NaN
2,607,0000001442024090920250612,0004187212024090920250612,Pressano Piazza,False,None,NaN
3,613,0000001482024090920250612,0004187222024090920250612,Pressano Piazza,False,None,NaN
4,610,0000001462024090920250612,0004187232024090920250612,Lavis Stazione Fs,False,None,NaN


In [13]:
merged_tickets = tickets.merge(trips, on='trip_id')
merged_tickets.head()

,count_fare,trip_id,day,timestamp,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,212,0004247422025030620250612,2025-05-16,2025-05-16 18:31:00,404,0000000532025030620250612,Guardini Centochiavi,True,D670_T0892_Ritorno_PRINCIPALE_ALTA_sub1,1.0
1,10,0004246692025030620250612,2025-05-16,2025-05-16 14:28:00,404,0000000102025030620250612,Guardini Centochiavi,True,D670_T0892_Ritorno_PRINCIPALE_ALTA_sub1,1.0
2,6,0004206572024090920250612,2025-05-16,2025-05-16 13:17:00,400,0000000052024090920250612,Piazza Dante Dogana,True,D170_F0512_Ritorno_sub1,1.0
3,80,0004229852024090920250612,2025-05-16,2025-05-16 16:01:00,594,0000001712024090920250612,Corso Verona Ospedale,True,D461_R5514a_Ritorno_PRINCIPALE_sub2,1.0
4,91,0004199532024090920250612,2025-05-16,2025-05-16 08:28:00,400,0000000102024090920250612,Piazza Dante Dogana,True,D612_T0542a_Ritorno_sub2,1.0


In [4]:
conn.rollback()

query = """
select trip_id, timestamp, traffic_level, normal, traffic
from traffic
"""
traffic = pd.read_sql(query, conn)
traffic.head()

,trip_id,timestamp,traffic_level,normal,traffic
0,0004188912024090920250612,2025-05-16 16:17:00,no traffic/low,778,836
1,0004190132024090920250612,2025-05-16 07:04:00,no traffic/low,905,926
2,0004190322024090920250612,2025-05-16 07:38:00,no traffic/low,777,807
3,0004190342024090920250612,2025-05-16 07:42:00,no traffic/low,745,796
4,0004193602024090920250612,2025-05-16 06:46:00,no traffic/low,909,877


In [14]:
traffic_merged = traffic.merge(trips, on='trip_id')
traffic_merged.head()

,trip_id,timestamp,traffic_level,normal,traffic,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,0004188912024090920250612,2025-05-16 16:17:00,no traffic/low,778,836,400,0000000102024090920250612,Piazza Dante Dogana,True,D170_F0512_Ritorno_sub1,1.0
1,0004190132024090920250612,2025-05-16 07:04:00,no traffic/low,905,926,404,0000000032024090920250612,Verona Palazzo Stella,False,D660_T0865_Andata_sub1,1.0
2,0004190322024090920250612,2025-05-16 07:38:00,no traffic/low,777,807,400,0000000102024090920250612,Povo Polo Sociale Cap.,False,D599_T0521h_Andata_sub1,1.0
3,0004190342024090920250612,2025-05-16 07:42:00,no traffic/low,745,796,404,0000000032024090920250612,Piazza Dante Dogana,True,D179_F0842b_Ritorno_ALTA_sub1,1.0
4,0004193602024090920250612,2025-05-16 06:46:00,no traffic/low,909,877,400,0000000022024090920250612,Oltrecastello,False,D593_T0501b_Andata_PRINCIPALE_sub2,1.0


In [15]:
traffic_tickets = traffic_merged.merge(merged_tickets, on='shape_id', how='inner')
traffic_tickets.head()
print(traffic_tickets.shape)

(718, 20)


Before aggregating with traffic we need to make sure that the shape_id is included inside the table so to do we need to join the table with the trips table on the trip_id column to get the shape_id column.

In [26]:
conn.rollback()

query = """
select *
from weather
"""
weather = pd.read_sql(query, conn)
weather = weather.rename(columns={'timestamp':'time'})
weather.head()

,measurement_id,time,latitude,longitude,weather_code,precipitation_probability,temperature,hour
0,5881,2025-05-27 18:31:00,46.04,11.119999,3,0.0,12.7,2025-05-15 00:00:00
1,5882,2025-05-27 18:31:00,46.04,11.119999,2,0.0,11.5,2025-05-15 01:00:00
2,5883,2025-05-27 18:31:00,46.04,11.119999,1,0.0,11.8,2025-05-15 02:00:00
3,5884,2025-05-27 18:31:00,46.04,11.119999,2,0.0,10.1,2025-05-15 03:00:00
4,5885,2025-05-27 18:31:00,46.04,11.119999,2,0.0,10.8,2025-05-15 04:00:00


In [27]:
traffic_tickets['timestamp'] = pd.to_datetime(traffic_tickets['timestamp_x'])
weather['hour'] = pd.to_datetime(weather['hour'])

traffic_tickets = traffic_tickets.sort_values('timestamp')
weather = weather.sort_values('hour')

In [28]:
merged = pd.merge_asof(
    traffic_tickets,
    weather[['hour', 'temperature', 'precipitation_probability', 'weather_code', 'latitude', 'longitude']],
    left_on='timestamp',
    right_on='hour',          # match on timestamp column
    direction='backward',     # can also use 'backward' or 'forward'
    tolerance=pd.Timedelta('1h')  # optional: max time difference allowed
)

ValueError: Merge keys contain null values on left side

In [22]:
merged

,trip_id,timestamp_x,traffic_level,normal,traffic,count_fare,day,timestamp_y,timestamp,hour,temperature,precipitation_probability,weather_code,latitude,longitude
0,0004229912024090920250612,2025-05-16 06:11:00,no traffic/low,460,498,35,2025-05-31,2025-05-31 06:23:00,2025-05-16 06:11:00,2025-05-16 06:00:00,12.4,0.0,0,46.04,11.119999
1,0004229912024090920250612,2025-05-16 06:11:00,no traffic/low,460,498,35,2025-05-16,2025-05-16 06:23:00,2025-05-16 06:11:00,2025-05-16 06:00:00,12.4,0.0,0,46.04,11.119999
2,0004229912024090920250612,2025-05-16 06:11:00,no traffic/low,460,498,35,2025-05-17,2025-05-17 06:23:00,2025-05-16 06:11:00,2025-05-16 06:00:00,12.4,0.0,0,46.04,11.119999
3,0004229912024090920250612,2025-05-16 06:11:00,no traffic/low,460,498,35,2025-05-18,2025-05-18 06:23:00,2025-05-16 06:11:00,2025-05-16 06:00:00,12.4,0.0,0,46.04,11.119999
4,0004229912024090920250612,2025-05-16 06:11:00,no traffic/low,460,498,35,2025-05-19,2025-05-19 06:23:00,2025-05-16 06:11:00,2025-05-16 06:00:00,12.4,0.0,0,46.04,11.119999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,0004210772024090920250612,2025-05-16 21:35:00,no traffic/low,909,734,17,2025-06-01,2025-06-01 22:06:00,2025-05-16 21:35:00,2025-05-16 21:00:00,11.8,0.0,1,46.04,11.119999
344,0004210772024090920250612,2025-05-16 21:35:00,no traffic/low,909,734,17,2025-05-18,2025-05-18 22:06:00,2025-05-16 21:35:00,2025-05-16 21:00:00,11.8,0.0,1,46.04,11.119999
345,0004210772024090920250612,2025-05-16 21:35:00,no traffic/low,909,734,17,2025-05-17,2025-05-17 22:06:00,2025-05-16 21:35:00,2025-05-16 21:00:00,11.8,0.0,1,46.04,11.119999
346,0004210772024090920250612,2025-05-16 21:35:00,no traffic/low,909,734,17,2025-05-27,2025-05-27 22:06:00,2025-05-16 21:35:00,2025-05-16 21:00:00,11.8,0.0,1,46.04,11.119999
